# Examples of usage:

In [ ]:
import os,sys
sys.path.append(os.path.abspath(".."))
from tool_box import tool_box
import pandas as pd
from pandas import DataFrame
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sb

%matplotlib inline

###  DICOM to nrrd conversion

In [ ]:
##Convert DICOM dataset to the volume (nrrd) format
# export_path = ... export path where the converted nrrds will be placed, region_of_interest = ... if you know exact name of
# the ROI you want to extract then write it with the ! character infront, eg. region_of_interest = !gtv1 , 
# if you want to extract all the gtv's in the rtstructure eg. gtv1, gtv2, gtv_whatever then just specify the stem word 
# eg. region_of_interest = gtv, default value is region_of_interest ='all', where all ROI's in rtstruct will be extracted.
        

In [ ]:
# Set up the parameters for the 
parameters = {'data_path': r'../data/dcms/', #path_to_your_data
              'data_type': 'dcm',
              'multi_rts_per_pat': False}   #When False, it will look only for 1 rtstruct in the patient folder, this will
                                            #speed up the process, if you have more then 1 rtstruct per patient, set it to True
                  
export_path =r'../data/'    

In [ ]:
#Inintialize the dataset 
data_ct = tool_box(**parameters) 

In [ ]:
#Run the conversion
data_ct.convert_to_nrrd(export_path,'gtv')

### Quick check of the ROI's in the nrrd dataset

In [ ]:
#If you want to check your converted images&ROIs or just any volumetric dataset (nrrd/mha), you can use get_jpegs function of
#the toolbox, it will generate jpeg images for each slice with the image and overlap of the contour.

In [ ]:
data_ct_nrrd = tool_box(data_path = r'../data/converted_nrrds/',
                    data_type='nrrd')

In [ ]:
data_ct_nrrd.get_jpegs(r'../data/')

In [ ]:
#Lets just check one of the patients

In [ ]:
from ipywidgets import interact
import numpy as np
from PIL import Image

def browse_images(images,names):
    n = len(images)
    def view_image(i):
        plt.figure(figsize=(20,10))
        plt.imshow(images[i])#, cmap=plt.cm.gray_r, interpolation='nearest')
        plt.title('Slice: %s' % names[i])
        plt.axis('off')
        plt.show()
    interact(view_image, i=(0,n-1))
    
for pat,_ in data_ct_nrrd:
    _,file_struct = [*os.walk(os.path.join('../data/images_quick_check/',pat))]
    root,images = file_struct[0],file_struct[2]
    imgs =[np.array(Image.open(os.path.join(root,img))) for img in images]
    print(pat)
    browse_images(imgs,images)
    break


    


### Extract PyRadiomics features

In [ ]:
data_ct_nrrd = tool_box(data_path = r'../data/converted_nrrds/',
                    data_type='nrrd')

In [ ]:
#Use your own pyradiomics parameters file, more you can find here: https://github.com/Radiomics/pyradiomics 
parameters = r"../example_ct_parameters.yaml"
features = data_ct_nrrd.extract_features(parameters,loggenabled=True)

In [ ]:
features.head(5)

In [ ]:
#Write features to excel file
writer = pd.ExcelWriter('../data/features/extracted_features.xlsx')
features.to_excel(writer,'Sheet1')
writer.save()

### Explore the dataset

In [ ]:
# Set up the parameters for the 
parameters = {'data_path': r'../data/dcms/', #path_to_your_data
              'data_type': 'dcm'}    

In [ ]:
data_dcms = tool_box(**parameters)
dataset_description = data_dcms.get_dataset_description() 

In [ ]:
dataset_description.head(3)

In [ ]:
# For example you can check for reduntant files by looking at unique modality values in your dataset
print('Unique modalities found: ',np.unique(dataset_description.Modality.values)[0])

In [ ]:
##There is two pre-set parameter dictionaries for 'CT' and 'MRI', but you can also send a custom list, 
#in this case the dicom tags should have the same names as corresponding pydicom (DICOM keyword, https://pydicom.github.io/pydicom/dev/old/base_element.html)
ct_dcms = tool_box(**parameters)
dataset_description = ct_dcms.get_dataset_description('CT') 

In [ ]:
dataset_description.head(3)

In [ ]:
#Lets plot some of the scans parameters

sb.set(context='poster', style='whitegrid')

study_date = sorted([ 'Nan' if x=='' or x=='NaN' else str(x[0:4]) for x in list(dataset_description['SeriesDate'])])[2:]
conv_kernel =['Nan' if x=='' or x=='NaN' else x for x in list(dataset_description['ConvolutionKernel'])]
tube_current =[-1 if x=='' or x=='NaN' else x for x in list(dataset_description['XRayTubeCurrent'])]
exposure =[-1 if x=='' or x=='NaN' else x for x in list(dataset_description['Exposure'])]
ps = sorted([(x[0]) for x in list(filter(lambda x: x != 'NaN', dataset_description['PixelSpacing'].values))])
sl_th = sorted([str(x)[0:3] for x in list(filter(lambda x: x != 'NaN', dataset_description['SliceThickness'].values))])
figures,descriptions = [study_date,conv_kernel,tube_current,exposure,ps,sl_th],['Study Date','Conv Kernel','Tube Current','Exposure','Pixel spacing','Slice Thickness']

fig,ax = plt.subplots(2,3,figsize=(25,15))
for i in range(2):
    for j in range(3):
        ax[i,j].hist(figures.pop(0),alpha=0.7)
        ax[i,j].set_title(descriptions.pop(0),fontsize=20)
    
    
